In [3]:
import pandas as pd

In [4]:
data = pd.read_csv("data_typeA.csv")
data.head()

,_id,CODE_REF,CODE_STA,STA_TH,PROVINCE,AMPHOE,TAMBON,LAT,LONG,X,...,HUMIDITY,WIND_SPEED,RAINFALL,RAINDAY,DEW_POINT,FEELS_LIKE,PRESSURE,CLOUDS,PM10,PM25
0,1,FON,NaN,สถานีตรวจวัดสภาพอากาศปางหินฝน,เชียงใหม่,แม่แจ่ม,ปางหินฝน,18.507271,98.229169,418632.09,...,68.84,1.58,56.01,5.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2,FON,NaN,สถานีตรวจวัดสภาพอากาศปางหินฝน,เชียงใหม่,แม่แจ่ม,ปางหินฝน,18.507271,98.229169,418632.09,...,87.19,2.01,275.22,17.0,NaN,NaN,NaN,NaN,NaN,NaN
2,3,FON,NaN,สถานีตรวจวัดสภาพอากาศปางหินฝน,เชียงใหม่,แม่แจ่ม,ปางหินฝน,18.507271,98.229169,418632.09,...,69.47,1.11,0.00,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,4,FON,NaN,สถานีตรวจวัดสภาพอากาศปางหินฝน,เชียงใหม่,แม่แจ่ม,ปางหินฝน,18.507271,98.229169,418632.09,...,84.86,1.11,175.23,18.0,NaN,NaN,NaN,NaN,NaN,NaN
4,5,FON,NaN,สถานีตรวจวัดสภาพอากาศปางหินฝน,เชียงใหม่,แม่แจ่ม,ปางหินฝน,18.507271,98.229169,418632.09,...,84.47,0.83,85.71,13.0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers

# โหลด dataset ใหม่

# เลือกฟีเจอร์ที่เกี่ยวข้อง
features = ['HUMIDITY', 'WIND_SPEED', 'RAINFALL', 'RAINDAY']  # เพิ่มฟีเจอร์ที่ต้องการ
target = 'RAINFALL'  # ใช้ RAINFALL เป็นเป้าหมาย

# สร้าง target label ว่าฝนตกหรือไม่ (1 = ฝนตก, 0 = ไม่ตก)
data['RAIN'] = np.where(data['RAINFALL'] > 0, 1, 0)

# แยกข้อมูลเป็นฟีเจอร์และเป้าหมาย
X = data[features].values
y = data['RAIN'].values

# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize ข้อมูล
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# สร้างโมเดล
model = keras.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))  # ใช้ sigmoid สำหรับ binary classification

# คอมไพล์โมเดล
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ฝึกโมเดล
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=1)

# ประเมินโมเดล
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy:.2f}')

# บันทึกโมเดล
model.save('model.h5')

# ฟังก์ชันสำหรับการพยากรณ์
def predict_rain(model, scaler, input_data):
    input_data_scaled = scaler.transform([input_data])
    prediction = model.predict(input_data_scaled)
    probability = prediction[0][0] * 100  # แปลงเป็นเปอร์เซ็นต์
    return probability

# ตัวอย่างการใช้งาน
example_input = [75, 2.5, 5.0, 0]  # ตัวอย่างข้อมูล (HUMIDITY, WIND_SPEED, RAINFALL, RAINDAY)
rain_probability = predict_rain(model, scaler, example_input)
print(f'Probability of rain: {rain_probability:.2f}%')


Epoch 1/50


c:\Users\hoker\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


187/187 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9215 - loss: 0.6079
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step - accuracy: 0.9520 - loss: 0.2673
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 854us/step - accuracy: 0.9529 - loss: 0.1951
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 905us/step - accuracy: 0.9540 - loss: 0.1872
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9486 - loss: 0.2026
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 868us/step - accuracy: 0.9562 - loss: 0.1801
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 896us/step - accuracy: 0.9484 - loss: 0.2036
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 891us/step - accuracy: 0.9549 - loss: 0.1840
Epoch 9/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 888us/step - accuracy: 0.9523 - loss: 0.1917
Epoch 10/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 919us/step - accuracy: 0.9534 - loss: 0.1884
Epoch 11/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 926us/step - accuracy: 0.9492 - loss: 0.2011
Epoch 12/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 0s 1

Accuracy: 0.95
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Probability of rain: 95.57%
